In [ ]:
!python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py train /u/horlavanasta/MasterProject//DataAndExperiments/Experiments/Experiments-1.5T-3T/NNs_Bayesian/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-WeightedCrossEntropy_augmTrue --n_splits 1 --split 0  --batch_size 5               

In [ ]:
def available_scans(source_dir, ext=".pt"):
    import os
    subject_ids=[]
    for (dirpath, dirnames, filenames) in os.walk(source_dir):
        for f in filenames:
            if f.endswith(ext):
                subject_ids.append(f)
    return subject_ids

In [17]:
with open("scripts/submit_experiments.sh", "r") as f:
    lines=f.readlines()
new_lines=[]
for i in range(1, len(lines)):
    if "WeightedCrossEntropy" not in lines[i]:
        new_lines.append("sbatch run_experiment.sh "+lines[i][:-1]+" True\n")
        
with open("scripts/submit_experiments.sh", "w") as f:
    for line in new_lines:
        f.write(line)

In [18]:
with open("scripts/submit_experiments-Copy1.sh", "r") as f:
    lines=f.readlines()
new_lines=[]    
for i in range(1, len(lines)):
    if "WeightedCrossEntropy" in lines[i]:
        new_lines.append(lines[i])  
        
with open("scripts/submit_experiments.sh", "a") as f:
    for line in new_lines:
        f.write(line)

In [5]:
f="/u/horlavanasta/MasterProject//DataAndExperiments/Experiments_3-fold/Experiments-3T/NNs_Bayesian/Conv5_FC3/subject_model-Conv5_FC3_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmTrue_20210528_134641"

CAPS_DIR="$HOME/MasterProject/DataAndExperiments/Data/CAPS"
NN_FOLDER="NNs_Bayesian"
TSV_PATH="$HOME/MasterProject/DataAndExperiments/Experiments_3-fold/Experiments-3T/labels/test"
OUTPUT_DIR="$HOME/MasterProject//DataAndExperiments/Experiments_3-fold/Experiments-1.5T/NNs_Bayesian/Conv5_FC3/"

POSTFIX="test_1.5T"
BAYESIAN=True
NBR_BAYESIAN_ITER=10

!python /u/horlavanasta/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py classify $CAPS_DIR $TSV_PATH $f $POSTFIX --bayesian $BAYESIAN --nbr_bayesian_iter $NBR_BAYESIAN_ITER --selection_metrics balanced_accuracy loss last_checkpoint


TEST_POSTFIX="test_3T"
TEST_TSV_PATH="$HOME/MasterProject/DataAndExperiments/Experiments_3-fold/Experiments-3T/labels/"
!python /u/horlavanasta/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py classify $CAPS_DIR $TEST_TSV_PATH $f $TEST_POSTFIX --bayesian $BAYESIAN --nbr_bayesian_iter $NBR_BAYESIAN_ITER --selection_metrics balanced_accuracy loss last_checkpoint --baseline False

In [6]:

folders=[f.path for f in os.scandir("/u/horlavanasta/MasterProject/DataAndExperiments/Data/CAPS_pt/subjects/")]
print(len(folders))

983


In [3]:
import pathlib
import pandas as pd
import os
import json
from clinicaaddl.visualize.plot import plot_generic

folders = []
MS_main_list = ['1.5T', "1.5T-3T"]
MS_list_dict = {'1.5T':['1.5T', '3T'], "3T": ['3T', '1.5T'], "1.5T-3T": ["1.5T-3T"]}
home_folder='/u/horlavanasta/MasterProject/'

isBayesian=True
for MS in MS_main_list[:]:
    print("MS ____________________________________________________________________________________________")
    model_types = [ "ResNet18", "SEResNet18", "ResNet18Expanded", "SEResNet18Expanded", "Conv5_FC3" ]
    MS_list = MS_list_dict[MS]
    inference_modes=["mode", "mean"]
    for inference_mode in inference_modes:
        results_folder_general =os.path.join(home_folder, 'Code/ClinicaTools/AD-DL/results/', "Experiments_3-fold", "Experiments_Bayesian" if isBayesian else "Experiments", "%s_inference"%inference_mode, 'Experiments-' + MS)
        model_dir_general = os.path.join(home_folder,"DataAndExperiments/Experiments_3-fold/Experiments-" + MS, "NNs_Bayesian" if isBayesian else "NNs")

        for network in model_types[:]:
            model_dir = os.path.join(model_dir_general, network)
            # output_dir = pathlib.Path(output_dir)
            modelPatter = "*default*"
            folders = [f for f in pathlib.Path(model_dir).glob(modelPatter)]

            for f in folders[:]:
                print(f)
                results_dir=os.path.join(results_folder_general,network)
#                 results_dir=None

#                 !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py bayesian $f stat
                !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir history uncertainty_distribution results --get_test_from_bayesian True --ba_inference_mode $inference_mode --catplot_type violinplot --uncertainty_metric "total_variance"
                !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir history uncertainty_distribution results --average_fold False --get_test_from_bayesian True --ba_inference_mode $inference_mode --catplot_type violinplot --uncertainty_metric "total_variance"

#             !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir history uncertainty_distribution results --get_test_from_bayesian True --catplot_type stripplot --uncertainty_metric "total_variance"
#             !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir uncertainty_catplot --catplot_type swarmplot --uncertainty_metric "entropy"
#             !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir uncertainty_catplot --catplot_type violinplot --uncertainty_metric "entropy"



MS ____________________________________________________________________________________________
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_3-fold/Experiments-1.5T/NNs_Bayesian/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmTrue_20210528_130119
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_3-fold/Experiments-1.5T/NNs_Bayesian/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmFalse_20210528_134446
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_3-fold/Experiments-1.5T/NNs_Bayesian/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmTrue
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_3-fold/Experiments-1.5T/NNs_Bayesian/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmFalse
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_3-fold/Experiments-1.5T/NNs_Bayesian/ResNet18

In [2]:
import seaborn as sns
print(sns.__version__)

0.10.0


In [ ]:
import pathlib
import pandas as pd
import os
import json
from clinicaaddl.visualize.plot import plot_generic

folders = []
MS_main_list = ['1.5T', "1.5T-3T"]
MS_list_dict = {'1.5T':['1.5T', '3T'], "3T": ['3T', '1.5T'], "1.5T-3T": ["1.5T-3T"]}
home_folder='/u/horlavanasta/MasterProject/'

isBayesian=True
for MS in MS_main_list[:]:
    print("MS ____________________________________________________________________________________________")
    model_types = [ "ResNet18", "SEResNet18", "ResNet18Expanded", "SEResNet18Expanded", "Conv5_FC3" ]
    MS_list = MS_list_dict[MS]
    inference_modes=["mode", "mean"]
    for inference_mode in inference_modes:
        results_folder_general =os.path.join(home_folder, 'Code/ClinicaTools/AD-DL/results/', "Experiments_3-fold", "Experiments_Bayesian" if isBayesian else "Experiments", "%s_inference"%inference_mode, 'Experiments-' + MS)
        model_dir_general = os.path.join(home_folder,"DataAndExperiments/Experiments_3-fold/Experiments-" + MS, "NNs_Bayesian" if isBayesian else "NNs")

        for network in model_types[:]:
            model_dir = os.path.join(model_dir_general, network)
            # output_dir = pathlib.Path(output_dir)
            modelPatter = "*default*"
            folders = [f for f in pathlib.Path(model_dir).glob(modelPatter)]

            for f in folders[:]:
                print(f)
                results_dir=os.path.join(results_folder_general,network)
#                 results_dir=None

#                 !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py bayesian $f stat
                !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir uncertainty_distribution results --get_test_from_bayesian True --ba_inference_mode $inference_mode --catplot_type violinplot --uncertainty_metric "total_variance"
                !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir uncertainty_distribution results --average_fold False --get_test_from_bayesian True --ba_inference_mode $inference_mode --catplot_type violinplot --uncertainty_metric "total_variance"

#             !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir history uncertainty_distribution results --get_test_from_bayesian True --catplot_type stripplot --uncertainty_metric "total_variance"
#             !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir uncertainty_catplot --catplot_type swarmplot --uncertainty_metric "entropy"
#             !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir uncertainty_catplot --catplot_type violinplot --uncertainty_metric "entropy"



MS ____________________________________________________________________________________________
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_3-fold/Experiments-1.5T/NNs_Bayesian/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmTrue_20210528_130119
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_3-fold/Experiments-1.5T/NNs_Bayesian/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmFalse_20210528_134446
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_3-fold/Experiments-1.5T/NNs_Bayesian/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmTrue
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_3-fold/Experiments-1.5T/NNs_Bayesian/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmFalse
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_3-fold/Experiments-1.5T/NNs_Bayesian/ResNet18

In [ ]:
import os
folders=[]
for f in os.scandir("/u/horlavanasta/MasterProject/DataAndExperiments/Data/CAPS_pt/subjects/"):
    if 'sub-' in f.path:
        pass
    folders.append(f.path)
print(len(folders))

In [ ]:
for f in folders:
    !echo $f
    !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py classify ~/MasterProject/ADNI_data/CAPSPreprocessedT1linear ~/MasterProject/ADNI_data/DataPrep/labels_new/test $f test